Combining All History JSON Files

In [ ]:
import pandas as pd
import time
#combining total history
history0 = pd.read_json('files/syd0.json')
history1 = pd.read_json('files/syd1.json')
history2 = pd.read_json('files/syd2.json')
history3 = pd.read_json('files/syd3.json')
history4 = pd.read_json('files/syd4.json')
history5 = pd.read_json('files/syd5.json')
history6 = pd.read_json('files/syd6.json')
history7 = pd.read_json('files/syd7.json')
history8 = pd.read_json('files/syd8.json')
history9 = pd.read_json('files/syd9.json')
history10 = pd.read_json('files/syd10.json')
history11 = pd.read_json('files/syd11.json')
history12 = pd.read_json('files/syd12.json')
history13 = pd.read_json('files/syd13.json')
history14 = pd.read_json('files/syd14.json')
history15 = pd.read_json('files/syd15.json')
history16 = pd.read_json('files/syd16.json')
history17 = pd.read_json('files/syd17.json')
history18 = pd.read_json('files/syd18.json')
history19 = pd.read_json('files/syd19.json')
complete = pd.concat([history0,history1,history2,history3,history4,history5,history6,history7,history8,history9,history10,history11,history12,history13,history14,history15,history16,history17,history18,history19])
print(complete.columns)







Index(['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='object')


Cleaning and Reformatting/naming Columns

In [2]:
#Drop unecessary columns and rename original
complete = complete.drop(columns=['platform', 'ip_addr', 'audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'offline', 'offline_timestamp'])
complete = complete.rename(columns={'ts':'exactTime', 'ms_played':'ms', 'conn_country':'country','master_metadata_track_name':'trackName',	'master_metadata_album_artist_name':'artistName','master_metadata_album_album_name':'albumName'	,'spotify_track_uri':'trackURI', 'episode_name':'episodeName','episode_show_name':'showName', 'spotify_episode_uri':'episodeURI','reason_start':'startReason','reason_end':'endReason','incognito_mode':'incognito'})

#convert ms to minutes
complete['duration'] = complete['ms']/60000
#extract specific times
complete['exactTime'] = pd.to_datetime(complete['exactTime'])
complete['hour'] = complete['exactTime'].dt.hour
complete['day'] = complete['exactTime'].dt.date
complete['month'] = complete['exactTime'].dt.month
complete['weekday'] = complete['exactTime'].dt.weekday
complete['year'] = complete['exactTime'].dt.year

complete.head() # complete['year'] = complete['year'].dt.year


,exactTime,ms,country,trackName,artistName,albumName,trackURI,episodeName,showName,episodeURI,...,endReason,shuffle,skipped,incognito,duration,hour,day,month,weekday,year
0,2020-04-06 20:27:23+00:00,21873,US,ball w/o you,21 Savage,i am > i was,spotify:track:50a8bKqlwDEqeiEknrzkTO,None,None,None,...,endplay,False,False,False,0.364550,20,2020-04-06,4,0,2020
1,2020-04-06 20:29:19+00:00,1830,US,Far From You,Rich The Kid,BOSS MAN,spotify:track:4VV8svAvJBYCPRTRwl01Z5,None,None,None,...,endplay,False,False,False,0.030500,20,2020-04-06,4,0,2020
2,2020-04-06 20:29:23+00:00,3950,US,Stuck Together (feat. Lil Baby),Rich The Kid,BOSS MAN,spotify:track:5cHPAqi6V5JPMy0vqS4lK6,None,None,None,...,endplay,False,False,False,0.065833,20,2020-04-06,4,0,2020
3,2020-04-06 20:29:29+00:00,5700,US,Sick (feat. DaBaby),Rich The Kid,BOSS MAN,spotify:track:0BqyyCV8M6roYLRmNtiasC,None,None,None,...,endplay,False,False,False,0.095000,20,2020-04-06,4,0,2020
4,2020-04-06 20:29:37+00:00,7420,US,Not Sorry (feat. Nicki Minaj),Rich The Kid,BOSS MAN,spotify:track:69pWehVZg0u3o17imQuEVj,None,None,None,...,endplay,False,False,False,0.123667,20,2020-04-06,4,0,2020


Split Tracks and Episodes

In [3]:
#Split up dataframe into tracks and episodes/podcasts

episodes = complete[complete['episodeName'].notnull()] #save rows where there's an episodeName
tracks = complete[complete['trackName'].notnull()] #save rows where there's a trackName
print(len(episodes))
print(len(tracks))
print(len(complete))
#tracks = complete[~complete['trackName'].notnull()]



616
304945
305562


Store all rows with unique artists

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from dotenv import load_dotenv
import os
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
#only keep unique artists
uniqueArtists = tracks.drop_duplicates(subset=['artistName'])
uniqueArtists.head()

print(len(uniqueArtists))



5254


Connect to PostgreSQL DB

In [ ]:

from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2

 #connection to postgres db
conn_string = 'postgresql://postgres:' + os.getenv('postgres_pass')  + '@localhost:' + os.getenv('postgres_port') + '/Spotify'
db = create_engine(conn_string)
#validate if connection works
inspector = inspect(db)
print(inspector.get_table_names(schema='public'))
conn = db.connect()






Populate table in postgres with track history

In [ ]:

#Transfer tracks rows to sql table trackhistory
tracks.to_sql('trackhistory', con=conn, if_exists='replace', index=False)
#Validate rows
print(pd.read_sql("SELECT COUNT(*) FROM trackhistory;", con=engine))




In [ ]:



# trackInfo = sp.track("spotify:track:50a8bKqlwDEqeiEknrzkTO") #Get spotify trackinfo
# artist = trackInfo['artists'][0] 
# artistID = artist['id'] #Get artist ID from artist objet
# genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
# print(genres)
#Dict to map artist with genres
# artistGenre = {}
# for t in uniqueArtists['trackURI']:
#     try: 
#         trackInfo = sp.track(t) #Get spotify trackinfo
#         artist = trackInfo['artists'][0] 
#         artistID = artist['id'] #Get artist ID from artist objet
#         genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
#         if artist['name'] not in artistGenre:
#             artistGenre[artist['name']] = genres
#         time.sleep(0.1)
#     except Exception as e: #If artist is invalid print error  ``
#         print(e)
#         continue
# print(len(artistGenre))
    
# #print(top)
# # print(client_id)
# # print(client_secret)

